In [1]:
%env CUDA_VISIBLE_DEVICES= 3

env: CUDA_VISIBLE_DEVICES=3


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

import tensorflow as tf

/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:


X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [ ]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')

In [ ]:
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [ ]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')



In [ ]:
W3 = tf.Variable(tf.random_normal([7*7*64,256], stddev = 0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)


In [ ]:
W4 = tf.Variable(tf.random_normal([256,10],stddev=0.01))
model = tf.matmul(L3, W4)

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

In [ ]:
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs= batch_xs.reshape(-1, 28,28,1)
        
        _, cost_val = sess.run([optimizer, cost],feed_dict = {X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch +1), 'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))

In [ ]:
correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy, feed_dict = {
    X: mnist.test.images.reshape(-1, 28,28,1),
    Y: mnist.test.labels
    
}))

In [ ]:
import tensorflow as tf

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

W3 = tf.Variable(tf.random_normal([7*7*64,256], stddev = 0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, dropout_prob)

W4 = tf.Variable(tf.random_normal([256,10],stddev=0.01))
model = tf.matmul(L3, W4)


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)


init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)


for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs= batch_xs.reshape(-1, 28,28,1)
        
        _, cost_val = sess.run([optimizer, cost],feed_dict = {X: batch_xs, Y: batch_ys, dropout_prob : 0.7})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch +1), 'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))
    
    
correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy, feed_dict = {
    X: mnist.test.images.reshape(-1, 28,28,1),
    Y: mnist.test.labels,
    dropout_prob : 1
}))

In [ ]:
import tensorflow as tf

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')

L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev = 0.01))
L3 = tf.nn.conv2d(L2, W3, strides = [1,1,1,1], padding = 'SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')


W4 = tf.Variable(tf.random_normal([4*4*128,256], stddev = 0.01))
L4 = tf.reshape(L3, [-1, 4*4*128])
L4 = tf.matmul(L4, W4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, dropout_prob)


W5 = tf.Variable(tf.random_normal([256,10],stddev=0.01))
model = tf.matmul(L4, W5)


cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)


init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)


for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs= batch_xs.reshape(-1, 28,28,1)
        
        _, cost_val = sess.run([optimizer, cost],feed_dict = {X: batch_xs, Y: batch_ys, dropout_prob : 0.7})
        total_cost += cost_val
        
    print('Epoch:', '%04d' % (epoch +1), 'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))
    
    
correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy, feed_dict = {
    X: mnist.test.images.reshape(-1, 28,28,1),
    Y: mnist.test.labels,
    dropout_prob : 1
}))

In [ ]:
import tensorflow as tf

X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

L1.shape

In [ ]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,5,5,1], padding = 'SAME')

L2.shape

In [ ]:
W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev = 0.01))
L3 = tf.nn.conv2d(L2, W3, strides = [1,1,1,1], padding = 'SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

L3.shape

In [ ]:
CUDA_VISIBLE_DEVICES = 3 jupyter notebook --ip '147.47.209.235'

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
batch_size = 100
gpu_num = 1
batch_size *= gpu_num
total_batch = int(mnist.train.num_examples/batch_size)

In [ ]:
def model(X, reuse = True):
    with tf.variable_scope('L1', reuse = reuse):
        L1 = tf.layers.conv2d(X, 32,[3,3], activation = 'relu', reuse = reuse)
        L1 = tf.layers.max_pooling2d(L1, [2,2],[2,2])
    with tf.variable_scope('L2', reuse = reuse):
        L2 = tf.layers.conv2d(L1,64,[3,3], activation = 'relu', reuse = reuse)
        L2 = tf.layers.max_pooling2d(L2, [2,2],[2,2])
    with tf.variable_scope('L3', reuse = reuse):
        L3 = tf.layers.conv2d(L2,64,[3,3], activation = 'relu', reuse = reuse)
        L3 = tf.layers.max_pooling2d(L3, [2,2],[2,2])
    with tf.variable_scope('L4', reuse = reuse):
        L4 = tf.layers.flatten(L3)
        L4 = tf.layers.dense(L4, 256, activation = 'relu')
        L4 = tf.layers.dropout(L4, dropout_prob)
        
    with tf.variable_scope('OUT', reuse= reuse):
        out = tf.layers.dense(L4, 10)
        
    return out

In [ ]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
dropout_prob = tf.placeholder(tf.float32)

In [ ]:
losses = []
Xs = tf.split(X, int(1))
Ys = tf.split(Y, int(1))

In [ ]:
for gpu_id in range(int(gpu_num)):
    with tf.device(tf.DeviceSpec(device_type="GPU",device_index = gpu_id)):
        with tf.variable_scope(tf.get_variable_scope(), reuse=(gpu_id>0)):
            cost = tf.nn.softmax_cross_entropy_with_logits_v2(
                logits=model(Xs[gpu_id], reuse=(gpu_id>0)),
                labels=Ys[gpu_id])
            losses.append(cost)
            
loss = tf.reduce_maen(tf.concat(losses, axis=0))